In [ ]:
# Run this in your first cell
!pip install psycopg2-binary pandas sqlalchemy geopandas folium plotly

In [ ]:
from google.colab import userdata

In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import os
from google.colab import userdata


# You'll need to store your database password/paramenters in Colab Secrets
DB_HOST = userdata.get('DB_HOST')
DB_PORT = userdata.get('DB_PORT')
DB_PASSWORD = userdata.get('DB_PASSWORD')
DB_USER = userdata.get("DB_USER")
DB_NAME = userdata.get('DB_NAME')


# Create connection string
connection_string = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

# Create SQLAlchemy engine
engine = create_engine(connection_string)

print("✅ Database connection established!")

In [ ]:
# Note: add in the msg_time column once the database has started including it
with engine.connect() as connection:
    captured_data = pd.read_sql_query(
        '''
        -- latest 100 entries give location back as latitude and longitude
        SELECT
          "vehicle_snapshots".*,
          ST_X("vehicle_snapshots"."location") AS longitude,
          ST_Y("vehicle_snapshots"."location") AS latitude
        FROM
          "vehicle_snapshots"
        ORDER BY
          "id" DESC
        NULLS LAST
        LIMIT
          100;
        ''', connection)

captured_data.head()

In [ ]:
captured_data.info()

# Parquet Export

In [ ]:
pip install pyarrow

In [ ]:
from datetime import datetime

In [ ]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
captured_data.to_parquet(f'abq_transit_data_{timestamp}.parquet', index=False)